# cooking the best possible web search result (in termes of relevance and speed)

In [1]:
user_query = "what is the latest top loser in the American stock market"

## test functions

In [2]:
from agentic_search.functions.web import get_serp_links

links = await get_serp_links(user_query)
links


[DEBUG] 
Search attempt failed: https://links.duckduckgo.com/d.js?q=what+is+the+latest+top+loser+in+the+American+stock+market&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-239432732350339170200444831947083665900&bing_market=wt-WT&ex=-1 202 Ratelimit

[DEBUG] 
serp results for query what is the latest top loser in the American stock market: []



[]

In [3]:
from agentic_search.functions.web import get_webpages_soups_text_async

res = await get_webpages_soups_text_async([x["href"] for x in links])
res

[]

## test chains

In [4]:
from agentic_search.chains.web import get_web_search_queries_chain

chain = get_web_search_queries_chain()
res = chain.invoke({"query": user_query})
res


{'queries': ['latest top loser stock market news',
  'worst performing stocks in US market today 2024-12-10',
  'American stock market worst performers this week',
  'top losing stocks in the S&P 500 index',
  'stock market losers in the Dow Jones Industrial Average',
  'latest stock market losses in the NASDAQ composite',
  'what are the biggest stock losers in the US market right now',
  'current top loser in the American stock exchange',
  'worst performing stocks in the US market this year 2024',
  'American stock market worst performers by sector',
  'top losing stocks in the US market by industry']}

In [5]:
from agentic_search.chains.web import get_web_search_query_chain

chain = get_web_search_query_chain()
res = chain.invoke({"query": user_query})
res

{'query': 'latest worst performing stocks in US market'}

## test tools

In [6]:
from agentic_search.tools.web import get_agentic_thorough_web_search_results_tool

res = await get_agentic_thorough_web_search_results_tool(user_query, 2)
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

[DEBUG] 
invoking thorough web search tool with query: what is the latest top loser in the American stock market

[DEBUG] 
Search attempt failed: https://links.duckduckgo.com/d.js?q=latest+top+loser+stock+market+news&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-314836786870172176860406889934307127589&bing_market=wt-WT&ex=-1 202 Ratelimit

[DEBUG] 
serp results for query latest top loser stock market news: []

[DEBUG] 
serp results for query worst performing stocks in the US today 2024-12-10: [{'title': 'Top Stock Losers Today - Yahoo Finance', 'href': 'https://finance.yahoo.com/screener/predefined/day_losers/', 'body': 'See the list of the top losing stocks today, including share price change and percentage, trading volume, intraday highs and lows, and day charts.'}, {'title': "2024's 10 Worst-Performing Stocks | Investing | U.S. News", 'href': 'https://money.usnews.com/investing/articles/worst-performing-stocks', 'body': 'Here are the 10 worst-performing stocks of 2024 among companies that trade

In [7]:
from agentic_search.tools.web import get_agentic_quick_web_search_results_tool

res = await get_agentic_quick_web_search_results_tool(user_query, 6)
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

[DEBUG] 
invoking quickweb search tool with query: what is the latest top loser in the American stock market

[DEBUG] 
serp results for query latest worst performing stocks in US market: [{'title': 'Top Stock Losers Today - Yahoo Finance', 'href': 'https://finance.yahoo.com/screener/predefined/day_losers/', 'body': 'See the list of the top losing stocks today, including share price change and percentage, trading volume, intraday highs and lows, and day charts.'}, {'title': 'Top Daily Losses: US stocks posting the largest losses today - Yahoo ...', 'href': 'https://finance.yahoo.com/markets/stocks/losers/', 'body': "Yahoo Finance's list of top losing stocks, includes share price changes, trading volume, intraday highs and lows, and day charts for stocks posting the largest losses today"}, {'title': 'Biggest Stock Losers in USA — TradingView', 'href': 'https://www.tradingview.com/markets/stocks-usa/market-movers-losers/', 'body': 'US companies below are the biggest stock losers today. Th

## test capabilities

In [8]:
from agentic_search.capabilities.web import get_web_search_results

res = await get_web_search_results("what is the ticker symbol of the biggest yoghurt company in the world")
res
# write results to file for consultation
with open("scratchpad.md", "w") as f:
    f.write(res)

[DEBUG] 
invoking web search agent with messages: [SystemMessage(content='You are a precise research assistant with web search capabilities. Your tasks:\n1. Provide accurate, current information\n2. Synthesize multi-source information concisely\n3. Include citations and maintain objectivity\n\nFocus on authoritative, verifiable sources only.\n\nSkip web search if you are completely certain of information.\nOtherwise, perform targeted searches as needed.\nToday is 2024-12-10.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the ticker symbol of the biggest yoghurt company in the world', additional_kwargs={}, response_metadata={}, id='47f87daf-5fc0-4205-9b33-be2daf184142')]

[DEBUG] 
invoking quickweb search tool with query: biggest yoghurt company in the world ticker

[DEBUG] 
Search attempt failed: https://links.duckduckgo.com/d.js?q=largest+yogurt+manufacturers+stock+symbol&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-83488218932653251007627598551352789787&bing_market